# 2 Backpropagation

See project 1 text for a more detailed description of this task. Also see associated PDF report.

Import relevant modules:

In [1]:
import torch
from torch import nn
from tests_backpropagation import main_test

# set seed and default data type
torch.manual_seed(123)
torch.set_default_dtype(torch.double)

## Class ``MyNet``

##### NOTE: Amalie has edited this to fit what is actually in MyNet

Read carefully how ``MyNet`` is implemented in the cell below. In particular:  
- ``n_l`` is a list of integers, representing the number units in each layer (including input layer).
-  ``MyNet([2, 3, 2]) = MiniNet()`` where ``MiniNet`` is the neural network defined in the fourth tutorial, in which notations are also clarified.     
- ``model.L`` is the number of layers, ``L`` (countes as the hidden layers and the output layer)
- ``model.f[l]`` is the activation function of layer ``l``, $f^{[l]}$ (here ``torch.tanh``)   
- ``model.df[l]`` is the derivative of the activation function, $f'^{[l]}$   
- ``model.a[l]``  is the tensor $A^{[l]}$, (shape: ``(1, n(l))``)   
- ``model.z[l]``  is the tensor $Z^{[l]}$, (shape: ``(1, n(l))``)  
- Weights $W^{[l]}$ (shape: ``(n(l+1), n(l))``) and biases $\mathbf{b}^{[l]}$ (shape: ``(n(l+1))``) can be accessed as follows:
```
weights = model.fc[str(l)].weight.data
bias = model.fc[str(l)].bias.data
```

In [2]:
# MLP - multi layer perception
class MyNet(nn.Module):
    def __init__(self, n_l = [2, 3, 2]):
        super().__init__() 
        
        # number of layers in our network (following Andrew's notations)
        self.L = len(n_l) - 1
        self.n_l = n_l
        
        # Where we will store our neuron values
        # - z: before activation function 
        # - a: after activation function (a = f(z))
        self.z = {i : None for i in range(1, self.L+1)}
        self.a = {i : None for i in range(self.L+1)}

        # Where we will store the gradients for our custom backpropagation algorithm
        self.dL_dw = {i : None for i in range(1, self.L+1)}
        self.dL_db = {i : None for i in range(1, self.L+1)}

        # Our activation functions in layers 1 through L (tanh)
        self.f = {i : lambda x : torch.tanh(x) for i in range(1, self.L+1)}

        # Derivatives of our activation functions in layers 1 through L
        self.df = {i : lambda x : (1/(torch.cosh(x)**2)) for i in range(1, self.L+1)}
        
        # Fully connected layers
        # We have to use nn.ModuleDict and to use strings as keys here to 
        # respect pytorch requirements (otherwise, the model does not learn)
        self.fc = nn.ModuleDict({str(i): None for i in range(1, self.L+1)})
        for i in range(1, self.L+1):
            self.fc[str(i)] = nn.Linear(in_features=n_l[i-1], out_features=n_l[i])
        
        
    def forward(self, x):
        # Input layer
        self.a[0] = torch.flatten(x, 1)
        
        # Hidden layers until output layer
        for i in range(1, self.L+1):

            # fully connected layer
            self.z[i] = self.fc[str(i)](self.a[i-1])
            # activation
            self.a[i] = self.f[i](self.z[i])

        # return output
        return self.a[self.L]

## Tasks

Write a function ``backpropagation(model, y_true, y_pred)`` that computes:

- $\frac{\partial L}{\partial w^{[l]}_{i,j}}$ and store them in ``model.dL_dw[l][i,j]`` for $l \in [1 .. L]$ 
- $\frac{\partial L}{\partial b^{[l]}_{j}}$ and store them in ``model.dL_db[l][j]`` for $l \in [1 .. L]$ 

assuming ``model`` is an instance of the ``MyNet`` class.


We use the mean squared error (MSE) for the loss function (notation for one training example):
\begin{equation}
    \mathcal{L}(\theta)=||\mathrm{\mathbf{y}}-\mathrm{\mathbf{\hat{y}}}||^2_2
\end{equation}
where $\theta$ is all the parameters to be optimized, $\mathrm{\mathbf{y}}$ is expected/true labels, and $\mathrm{\mathbf{\hat{y}}}$ are the predicted labels. The loss function describes how far the outputs are from the expected results.

The weights are part of $\theta$, and the derivative for the weights is given in Equation (4) in the project text:
\begin{equation}
    \frac{\partial \mathcal{L}}{\partial w^{[l]}_{i,j}}=\delta^{[l]}_i\times a^{[l-1]}_j,
\end{equation}
where $\delta_i^{[l]}$ is the local gradient:
\begin{equation}
    \delta^{[l]}_i=\frac{\partial \mathcal{L}}{\partial z^{[l]}_i}=\frac{\partial \mathcal{L}}{\partial a^{[l]}_i}\times \frac{\partial a^{[l]}_i}{\partial z^{[l]}_i}
\end{equation}


Equation (5) shows the local derivative in the case for the output layer where $\mathrm{\mathbf{a}}^{[L]}=\mathrm{\mathbf{\hat{y}}}$:
    
\begin{equation}
    \delta^{[l]}_i=\frac{\partial \mathcal{L}}{\partial \hat{y}_i}\times \frac{\partial \hat{y}_i}{\partial z^{[L]}_i}=-2(y_i-\hat{y}_i)\times f'^{[L]}_i(z^{[L]}_i)=2(\hat{y}_i-y_i)\times f'^{[L]}_i(z^{[L]}_i)
\end{equation}

And Equation (6) whos the local gradient for the general case:
\begin{equation}
    \delta^{[l]}_i=\frac{\partial \mathcal{L}}{\partial a^{[l]}_i}\times \frac{\partial a^{[l]}_i}{\partial z^{[l]}_i}=\left(\sum_{k=1}^{n^{[l+1]}} \frac{\partial \mathcal{L}}{\partial a^{[l+1]}_k}\frac{\partial a^{[l+1]}_k}{\partial a^{[l]}_i}\right)\times \frac{\partial a^{[l]}_i}{\partial z^{[l]}_i}=\left(\sum_{k=1}^{n^{[l+1]}} \delta^{[l+1]}_k w^{[l+1]}_{k,i}\right)\times f'^{[l]}_i(z^{[l]}_i)
\end{equation}

and we have that for the biases $\mathrm{\mathbf{b}}^{[l]}$ (also belonging to $\theta$):
\begin{equation}
    \frac{\partial \mathcal{L}}{\partial b^{[l]}_j}=\delta^{[l]}_j\times 1
\end{equation}

In the code, the derivatives are noted with underscore instead of fraction. An additional underscore is added if a specific layer is referenced. For example, $\frac{\partial \mathcal{L}}{\partial \mathbf{a}^{[L]}}$ is written as ``dL_da_L``. In addition, $f'^{[L]}(\mathbf{z}^{[L]})$ is denoted ``df_z_L``. Comments on dimensions are written as if $m=1$ (mini-batch) but the code is vectorized and works for training with multiple training examples ($m>1$). ``torch.einsum`` is used to ensure correct dimensions of the tensors when multiplying.

In [3]:
def backpropagation(model, y_true, y_pred):
    ''' 
    backpropagation using existing a, z, weight and bias from forward prop 
    calculates dL/dw and dL/db for all layers in [1,...,L] and updates 
    MyNet model
    '''

    ##### Last layer L is first in backprop
    
    dL_da_L = 2*torch.sub(y_pred, y_true) # n^[L] x 1, input for first backprop layer (dL/da, eq. 5)

    df_z_L = model.df[model.L](model.z[model.L]) # n^[L] x 1, da/dz=f'(z) for first backprop layer
    
    # local gradient in output layer - eq. 5 in project text
    delta_L = torch.einsum('ij, ij -> ij', dL_da_L, df_z_L) # n^[L] x 1

    # eq. 4 in project text
    dL_dw_L = torch.einsum('ij, ik -> jk', delta_L, model.a[model.L-1]) # n^[L] x n^[L-1]
    
    # update in model
    model.dL_dw[model.L] = dL_dw_L
    model.dL_db[model.L] = torch.squeeze(delta_L) # squeeze to drop extra dimension

    # save delta for backprop (eq. 6)
    delta_ps = delta_L
    
    #### loop through hidden layers, [1,...,L-1], from last to first
    for l in range(model.L-1, 0, -1): 
        
        # f'(z) for current layer - l
        df_z = model.df[l](model.z[l]) # n^[l] x 1
        
        # weights from previous backprop step (eq. 6)
        weights_ps = model.fc[str(l+1)].weight.data # n^[l+1] x n^[l]

        # eq. 6 in project text
        delta_current = torch.einsum('ij,jk,ik->ik', delta_ps, weights_ps, df_z)

        dL_dw = torch.einsum('ij,ik->jk', delta_current, model.a[l-1])
        
        # update in model
        model.dL_dw[l] = dL_dw
        model.dL_db[l] = torch.squeeze(delta_current)
        
        # save delta for further backprop (eq. 6)
        delta_ps = delta_current

## Run the cells below, and check the output

- In the 1st cell, we use a toy dataset and the same architecture as the MiniNet class of the fourth tutorial. 
- In the 2nd cell, we use a few samples of the MNIST dataset with a consistent model architecture (``24x24`` black and white cropped images as input and ``10`` output classes). 

You can set ``verbose`` to ``True`` if you want more details about your computations versus what is expected.

In [4]:
model = MyNet([2, 3, 2])
main_test(backpropagation, model, verbose=True, data='toy')


 ========================  Check gradients ======================== 

 ================= Epoch 1 ================= 

 ------------ fc['1'].weight.grad ------------ 
  Our computation:
 tensor([[ 6.7814e-08,  7.2355e-08],
        [-1.0720e-03, -1.1438e-03],
        [-2.2284e-03, -2.3776e-03]], grad_fn=<ViewBackward>)
  Autograd's computation:
 tensor([[ 6.7814e-08,  7.2355e-08],
        [-1.0720e-03, -1.1438e-03],
        [-2.2284e-03, -2.3776e-03]])

 ------------- fc['1'].bias.grad ------------- 
  Our computation:
 tensor([ 8.0769e-09, -1.2768e-04, -2.6541e-04], grad_fn=<SqueezeBackward0>)
  Autograd's computation:
 tensor([ 8.0769e-09, -1.2768e-04, -2.6541e-04])

 ------------- relative error ------------ 
(fc[1].weight.grad, model.dL_dw[1]):   0.0000
(fc[1].bias.grad, model.dL_db[1]):   0.0000
(fc[2].weight.grad, model.dL_dw[2]):   0.0000
(fc[2].bias.grad, model.dL_db[2]):   0.0000
Gradients consistent with finite differences computations. :) 

 ================= Epoch 2 =========

Tests passed for packrop implementation when testing with toy (random) data set.

In [6]:
model = MyNet([24*24, 16, 10])
main_test(backpropagation, model, verbose=False, data='mnist')


 ========================  Check gradients ======================== 

 ================= Epoch 1 ================= 
Gradients consistent with autograd's computations. :) 
Gradients consistent with finite differences computations. :) 

 ==============  Check that weights have been updated ============= 
Weights have been updated. :)

 ===================  Check computational graph =================== 
All parameters seem correctly attached to the computational graph! :) 


Tests passed for backprop implementation on model when testing with MNIST data set.